In [2]:
import re
import logging
from typing import Optional, Union

from df_engine.core.keywords import TRANSITIONS, RESPONSE
from df_engine.core import Context, Actor
import df_engine.conditions as cnd
from  scenario.condition import healmed_request
from language.dutch import dutch_transform
from language.spanish import spanish_transform
from language.greek import greek_transform
from language.russian import russian_transform
from language.danish import danish_transform
from language.arabic import arabic_transform

logger = logging.getLogger(__name__)

# First of all, to create a dialog agent, we need to create a dialog script.
# Below, `plot` is the dialog script.
# A dialog script is a flow dictionary that can contain multiple plot .
# Plot are needed in order to divide a dialog into sub-dialogs and process them separately.
# For example, the separation can be tied to the topic of the dialog.
# In our example, there is one flow called greeting_flow.

# Inside each flow, we can describe a sub-dialog.
# Here we can also use keyword `LOCAL`, which we have considered in other examples.

# Flow describes a sub-dialog using linked nodes, each node has the keywords `RESPONSE` and `TRANSITIONS`.

# `RESPONSE` - contains the response that the dialog agent will return when transitioning to this node.
# `TRANSITIONS` - describes transitions from the current node to other nodes.
# `TRANSITIONS` are described in pairs:
#      - the node to which the agent will perform the transition
#      - the condition under which to make the transition
plot = {
    "greeting_flow": {
        "start_node": {  # This is an initial node, it doesn't need an `RESPONSE`
            RESPONSE: "",
            TRANSITIONS: {("english", "node1"): cnd.regexp(r"\bHi\b|hi|hello|Hello|hey|Hey", re.IGNORECASE),
                          ("dutch", "node1"): cnd.regexp(r"\bHallo\b|hoi|hallo", re.IGNORECASE),
                          ("spanish", "node1"): cnd.regexp(r"\bHola\b|hola|oye|Oye", re.IGNORECASE),
                          ("greek", "node1"): cnd.regexp(r"\bγεια\b|γεια|γεια σας|Γεια σου|Χαίρετε|γεια|Γεια", re.IGNORECASE),
                          ("russian", "node1"): cnd.regexp(r"\bПривет\b|Здравствуйте|Привет|Эй", re.IGNORECASE),
                          ("danish", "node1"): cnd.regexp(r"\bHej\b|hej|hallo|Hallo|hey|Hey", re.IGNORECASE),
                          ("arabic", "node1"): cnd.regexp(r"\bأهلا\b|مهلا|أهلين|سلام", re.IGNORECASE)},
        },
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: "Sure Sir, I thanks for your visit but your can go other services if needed you can type for english 'hi', for dutch 'Hallo', for spanish 'Hola', for greek 'γεια, for russian 'Привет', for danish 'Hej' and for arabic 'أهلا'",
            TRANSITIONS: {("english", "node1"): cnd.regexp(r"\bHi\b|hi|hello|Hello|hey|Hey", re.IGNORECASE),
                          ("dutch", "node1"): cnd.regexp(r"\bHallo\b|hoi|hallo", re.IGNORECASE),
                          ("spanish", "node1"): cnd.regexp(r"\bHola\b|hola|Oye|oye", re.IGNORECASE),
                          ("greek", "node1"): cnd.regexp(r"\bγεια\b|γεια|γεια σας|Γεια σου|Χαίρετε|γεια|Γεια", re.IGNORECASE),
                          ("russian", "node1"): cnd.regexp(r"\bПривет\b|Здравствуйте|Привет|Эй", re.IGNORECASE),
                          ("danish", "node1"): cnd.regexp(r"\bHej\b|hej|hallo|Hallo|hey|Hey", re.IGNORECASE),
                          ("arabic", "node1"): cnd.regexp(r"\bأهلا\b|مهلا|أهلين|سلام|أهل", re.IGNORECASE)},
        },
    },
    "english": {
        "node1": {
            RESPONSE: "Hi!, I'm Health and medicine Chatbot Assistant, how may help you?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental_health|Mental_health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental_health|Mental_health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental_health|Mental_health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental_health|Mental_health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welcome Sir/Mam",
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental_health|Mental_health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE),
                         "node6": cnd.regexp(r"\bNo|no|NO|Nothing|nothing", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "bye", TRANSITIONS: {"node1": cnd.exact_match("Hi")}},
        
  },
    "dutch": {
        "node1": {
            RESPONSE: "Hallo!, ik ben Chatbot-assistent voor gezondheid en geneeskunde, hoe kan u helpen?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\bafspraak|Afspraak\b|\barts|Arts|Artsen|Cardiologen|Gastro-enterologen|Tandarts|KNO-arts|Gynaecoloog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bgezondheid|Gezondheid\b|\bgeneeskunde|Geneeskunde", re.IGNORECASE),
                         "node4": cnd.regexp(r"\beetpatroon|Eetpatroon\b|\bmentale_gezondheid|Mentale_gezondheid", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bBedankt|bedankt|Bedankt|bedankt|\bGoed|\bgoed|OK|ok|Ok|Met dank", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: dutch_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bafspraak|Afspraak\b|\barts|Arts|Artsen|Cardiologen|Gastro-enterologen|Tandarts|KNO-arts|Gynaecoloog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bgezondheid|Gezondheid\b|\bgeneeskunde|Geneeskunde", re.IGNORECASE),
                         "node4": cnd.regexp(r"\beetpatroon|Eetpatroon\b|\bmentale_gezondheid|Mentale_gezondheid", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bBedankt|bedankt|Bedankt|bedankt|Goed|goed|OK|ok|Ok|Met danks", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: dutch_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bafspraak|Afspraak\b|\barts|Arts|Artsen|Cardiologen|Gastro-enterologen|Tandarts|KNO-arts|Gynaecoloog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bgezondheid|Gezondheid\b|\bgeneeskunde|Geneeskunde", re.IGNORECASE),
                         "node4": cnd.regexp(r"\beetpatroon|Eetpatroon\b|\bmentale_gezondheid|Mentale_gezondheid", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bBedankt|bedankt|Bedankt|bedankt|Goed|goed|OK|ok|Ok|Met danks", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: dutch_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bafspraak|Afspraak\b|\barts|Arts|Artsen|Cardiologen|Gastro-enterologen|Tandarts|KNO-arts|Gynaecoloog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bgezondheid|Gezondheid\b|\bgeneeskunde|Geneeskunde", re.IGNORECASE),
                         "node4": cnd.regexp(r"\beetpatroon|Eetpatroon\b|\bmentale_gezondheid|Mentale_gezondheid", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bBedankt|bedankt|Bedankt|bedankt|Goed|goed|OK|ok|Ok|Met danks", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welkom meneer/mama",
            TRANSITIONS: {"node2": cnd.regexp(r"\bafspraak|Afspraak\b|\barts|Arts|Artsen|Cardiologen|Gastro-enterologen|Tandarts|KNO-arts|Gynaecoloog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bgezondheid|Gezondheid\b|\bgeneeskunde|Geneeskunde", re.IGNORECASE),
                         "node4": cnd.regexp(r"\beetpatroon|Eetpatroon\b|\bmentale_gezondheid|Mentale_gezondheid", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bBedankt|bedankt|Bedankt|bedankt|Goed|goed|OK|ok|Ok|Met danks", re.IGNORECASE),
                         "node6": cnd.regexp(r"\bNee|nee|NEE|Niets|niets", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "Natuurlijk meneer, ik dank u voor uw bezoek, maar u kunt indien nodig naar andere diensten gaan, u kunt 'hallo' typen", 
                  TRANSITIONS: {"node1": cnd.regexp(r"\bHallo\b|hoi|hallo", re.IGNORECASE)}
                 },
        
  },
    "spanish": {
        "node1": {
            RESPONSE: "Hola!, soy asistente de un chatbot de salud y medicina, cómo puedo ayudarte?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\bcita|Cita\b|\bmédica|médico|Médica|Médico|Médicos|Médicas|Cardiólogos|Gastroenterólogos|Dentista|Otorrinolaringólogo|Ginecólogo", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsalud|Salud|medicamento|Medicamento", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdieta|Dieta\b|\bsalud_mental|Salud_mental", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bGracias|gracias|Agradecimiento|agradecimiento|Bien|bien|OK|ok|Ok", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: spanish_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bcita|Cita\b|\bmédica|médico|Médica|Médico|Médicos|Médicas|Cardiólogos|Gastroenterólogos|Dentista|Otorrinolaringólogo|Ginecólogo", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsalud|Salud|medicamento|Medicamento", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdieta|Dieta\b|\bsalud_mental|Salud_mental", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bGracias|gracias|Agradecimiento|agradecimiento|Bien|bien|OK|ok|Ok", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: spanish_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bcita|Cita\b|\bmédica|médico|Médica|Médico|Médicos|Médicas|Cardiólogos|Gastroenterólogos|Dentista|Otorrinolaringólogo|Ginecólogo", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsalud|Salud|medicamento|Medicamento", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdieta|Dieta\b|\bsalud_mental|Salud_mental", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bGracias|gracias|Agradecimiento|agradecimiento|Bien|bien|OK|ok|Ok", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: spanish_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bcita|Cita\b|\bmédica|médico|Médica|Médico|Médicos|Médicas|Cardiólogos|Gastroenterólogos|Dentista|Otorrinolaringólogo|Ginecólogo", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsalud|Salud|medicamento|Medicamento", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdieta|Dieta\b|\bsalud_mental|Salud_mental", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bGracias|gracias|Agradecimiento|agradecimiento|Bien|bien|OK|ok|Ok", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welkom meneer/mama",
            TRANSITIONS: {"node2": cnd.regexp(r"\bcita|Cita\b|\bmédica|médico|Médica|Médico|Médicos|Médicas|Cardiólogos|Gastroenterólogos|Dentista|Otorrinolaringólogo|Ginecólogo", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsalud|Salud|medicamento|Medicamento", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdieta|Dieta\b|\bsalud_mental|Salud_mental", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bGracias|gracias|Agradecimiento|agradecimiento|Bien|bien|OK|ok|Ok", re.IGNORECASE),
                         "node6": cnd.regexp(r"\bNo|no|NO|Nada|nada", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "Claro señor, gracias por su visita, pero puede ir a otros servicios si es necesario, puede escribir 'hola'", 
                  TRANSITIONS: {"node1": cnd.regexp(r"\bHola\b|hola||oye|Oye", re.IGNORECASE)}
          }
      },
    "greek": {
        "node1": {
            RESPONSE: "Γεια!, είμαι Βοηθός Chatbot για την υγεία και την ιατρική, πώς μπορεί να σας βοηθήσει?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\bραντεβού|Ραντεβού\b|\bγιατρός|Γιατρός|Γιατροί|Καρδιολόγοι|Γαστρεντερολόγοι|Οδοντίατρος|Ειδικός ΩΡΛ|Γυναικολόγος", re.IGNORECASE),
                         "node3": cnd.regexp(r"\b^υγεία|^Υγεία|φάρμακο|Ιατρική", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bδίαιτα|Δίαιτα\b|\bψυχική_υγεία$|ψυχική_υγεία$|διατροφή", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bΕυχαριστώ|Ευχαριστώ|Ευχαριστώ|Ευχαριστώ|Καλά|καλά|Εντάξει|εντάξει|Εντάξει", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: greek_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bραντεβού|Ραντεβού\b|\bγιατρός|Γιατρός|Γιατροί|Καρδιολόγοι|Γαστρεντερολόγοι|Οδοντίατρος|Ειδικός ΩΡΛ|Γυναικολόγος", re.IGNORECASE),
                         "node3": cnd.regexp(r"\b^υγεία|^Υγεία|φάρμακο|Ιατρική", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bδίαιτα|Δίαιτα\b|\bψυχική_υγεία$|ψυχική_υγεία$|διατροφή", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bΕυχαριστώ|Ευχαριστώ|Ευχαριστώ|Ευχαριστώ|Καλά|καλά|Εντάξει|εντάξει|Εντάξει", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: greek_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bραντεβού|Ραντεβού\b|\bγιατρός|Γιατρός|Γιατροί|Καρδιολόγοι|Γαστρεντερολόγοι|Οδοντίατρος|Ειδικός ΩΡΛ|Γυναικολόγος", re.IGNORECASE),
                         "node3": cnd.regexp(r"\b^υγεία|^Υγεία|φάρμακο|Ιατρική", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bδίαιτα|Δίαιτα\b|\bψυχική_υγεία$|ψυχική_υγεία$|διατροφή", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bΕυχαριστώ|Ευχαριστώ|Ευχαριστώ|Ευχαριστώ|Καλά|καλά|Εντάξει|εντάξει|Εντάξει", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: greek_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bραντεβού|Ραντεβού\b|\bγιατρός|Γιατρός|Γιατροί|Καρδιολόγοι|Γαστρεντερολόγοι|Οδοντίατρος|Ειδικός ΩΡΛ|Γυναικολόγος", re.IGNORECASE),
                         "node3": cnd.regexp(r"\b^υγεία|^Υγεία|φάρμακο|Ιατρική", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bδίαιτα|Δίαιτα\b|\bψυχική_υγεία$|ψυχική_υγεία$|διατροφή", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bΕυχαριστώ|Ευχαριστώ|Ευχαριστώ|Ευχαριστώ|Καλά|καλά|Εντάξει|εντάξει|Εντάξει", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welkom meneer/mama",
            TRANSITIONS: {"node2": cnd.regexp(r"\bραντεβού|Ραντεβού\b|\bγιατρός|Γιατρός|Γιατροί|Καρδιολόγοι|Γαστρεντερολόγοι|Οδοντίατρος|Ειδικός ΩΡΛ|Γυναικολόγος", re.IGNORECASE),
                         "node3": cnd.regexp(r"\b^υγεία|^Υγεία|φάρμακο|Ιατρική", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bδίαιτα|Δίαιτα\b|\bψυχική_υγεία$|ψυχική_υγεία$|διατροφή", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bΕυχαριστώ|Ευχαριστώ|Ευχαριστώ|Ευχαριστώ|Καλά|καλά|Εντάξει|εντάξει|Εντάξει", re.IGNORECASE),
                         "node6": cnd.regexp(r"\bΌχι|όχι|ΟΧΙ|Τίποτα|τίποτα", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "Βεβαίως κύριε, ευχαριστώ για την επίσκεψή σας, αλλά μπορείτε να πάτε σε άλλες υπηρεσίες αν χρειαστεί μπορείτε να πληκτρολογήσετε 'γεια'", 
                  TRANSITIONS: {"node1": cnd.regexp(r"\bγεια\b|γεια|γεια σας|Γεια σου|Χαίρετε|γεια|Γεια", re.IGNORECASE)}
          }
      },
    "russian": {
        "node1": {
            RESPONSE: "Привет! Я Здоровье и медицина Помощник по чат-боту, как может вам помочь?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\bназначение|Назначение\b|\bврач|Доктор|Врачи|Кардиологи|Гастроэнтерологи|Дантист|ЛОР-специалист|Гинеколог", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bздоровье|Здоровье\b|\bлекарство", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bдиета|Диета\b|\bпсихическое_здоровье|Психическое_здоровье", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bСпасибо|спасибо|Благодарю|благодарю|Хорошо|хорошо|ОК|ок|Ok", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: russian_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bназначение|Назначение\b|\bврач|Доктор|Врачи|Кардиологи|Гастроэнтерологи|Дантист|ЛОР-специалист|Гинеколог", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bздоровье|Здоровье\b|\bлекарство", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bдиета|Диета\b|\bпсихическое_здоровье|Психическое_здоровье", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bСпасибо|спасибо|Благодарю|благодарю|Хорошо|хорошо|ОК|ок|Ok", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: russian_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bназначение|Назначение\b|\bврач|Доктор|Врачи|Кардиологи|Гастроэнтерологи|Дантист|ЛОР-специалист|Гинеколог", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bздоровье|Здоровье\b|\bлекарство", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bдиета|Диета\b|\bпсихическое_здоровье|Психическое_здоровье", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bСпасибо|спасибо|Благодарю|благодарю|Хорошо|хорошо|ОК|ок|Ok", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: russian_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bназначение|Назначение\b|\bврач|Доктор|Врачи|Кардиологи|Гастроэнтерологи|Дантист|ЛОР-специалист|Гинеколог", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bздоровье|Здоровье\b|\bлекарство", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bдиета|Диета\b|\bпсихическое_здоровье|Психическое_здоровье", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bСпасибо|спасибо|Благодарю|благодарю|Хорошо|хорошо|ОК|ок|Ok", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welcome Sir/Mam",
            TRANSITIONS: {"node2": cnd.regexp(r"\bназначение|Назначение\b|\bврач|Доктор|Врачи|Кардиологи|Гастроэнтерологи|Дантист|ЛОР-специалист|Гинеколог", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bздоровье|Здоровье\b|\bлекарство", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bдиета|Диета\b|\bпсихическое_здоровье|Психическое_здоровье", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bСпасибо|спасибо|Благодарю|благодарю|Хорошо|хорошо|ОК|ок|Ok", re.IGNORECASE),
                         "node6": cnd.regexp(r"\bНет|нет|НЕТ|Ничего|ничего", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "Конечно сэр, Я благодарю за ваш визит, но вы можете воспользоваться другими услугами, если это необходимо, вы можете ввести 'Привет'", 
                  TRANSITIONS: {"node1": cnd.regexp(r"\bПривет\b|Здравствуйте|Привет|Эй", re.IGNORECASE)}
          },
        
    },
    "danish": {
        "node1": {
            RESPONSE: "Hej!, jeg er Sundhed og Medicin Chatbot assistent, hvordan kan hjælpe dig?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\budnævnelse|Udnævnelse\b|\blæge|Læge|Læger|Kardiologer|Gastroenterologer|Tandlæge|ØNH-specialist|Gynækolog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsundhed|Sundhed\b|\bmedicin|Medicin", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiæt|Diæt\b|\bmental_sundhed|Mental_sundhed", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bTak|tak|Takker|takker|God|god|OK|ok|Ok", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: danish_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\budnævnelse|Udnævnelse\b|\blæge|Læge|Læger|Kardiologer|Gastroenterologer|Tandlæge|ØNH-specialist|Gynækolog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsundhed|Sundhed\b|\bmedicin|Medicin", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiæt|Diæt\b|\bmental_sundhed|Mental_sundhed", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bTak|tak|Takker|takker|God|god|OK|ok|Ok", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: danish_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\budnævnelse|Udnævnelse\b|\blæge|Læge|Læger|Kardiologer|Gastroenterologer|Tandlæge|ØNH-specialist|Gynækolog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsundhed|Sundhed\b|\bmedicin|Medicin", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiæt|Diæt\b|\bmental_sundhed|Mental_sundhed", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bTak|tak|Takker|takker|God|god|OK|ok|Ok", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: danish_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\budnævnelse|Udnævnelse\b|\blæge|Læge|Læger|Kardiologer|Gastroenterologer|Tandlæge|ØNH-specialist|Gynækolog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsundhed|Sundhed\b|\bmedicin|Medicin", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiæt|Diæt\b|\bmental_sundhed|Mental_sundhed", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bTak|tak|Takker|takker|God|god|OK|ok|Ok", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welcome Sir/Mam",
            TRANSITIONS: {"node2": cnd.regexp(r"\budnævnelse|Udnævnelse\b|\blæge|Læge|Læger|Kardiologer|Gastroenterologer|Tandlæge|ØNH-specialist|Gynækolog", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bsundhed|Sundhed\b|\bmedicin|Medicin", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiæt|Diæt\b|\bmental_sundhed|Mental_sundhed", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bTak|tak|Takker|takker|God|god|OK|ok|Ok", re.IGNORECASE),
                         "node6": cnd.regexp(r"\bNej|nej|NEJ|Intet|ingenting|ikke noget|Ikke noget", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "Klart herre, jeg takker for dit besøg men din kan gå andre tjenester hvis det er nødvendigt kan du skrive 'hej'", 
                  TRANSITIONS: {"node1": cnd.regexp(r"\bHej\b|hej|hallo|Hallo|hey|Hey", re.IGNORECASE)}
          },
        
  },
    "arabic": {
        "node1": {
            RESPONSE: "مرحبًا ، أنا مساعد الشات بوت الخاص بالصحة والطب ، كيف يمكن أن يساعدك!?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\bالتعيين|ميعاد\b|\bطبيب|دكتور|أطباء|أطباء القلب|أطباء الجهاز الهضمي|دكتور أسنانأخصائي الأنف والحنجرة|طبيب نساء وولادة", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bصحة\b|\bالطب|طب|دواء", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bالصحة_العقلية\b|\حمية غذائية|حمية", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bشكرًا|شكرًا|شكر|شكر|جيد|جيد|حسنًا|موافق|حسنًا", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: arabic_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bالتعيين|ميعاد\b|\bطبيب|دكتور|أطباء|أطباء القلب|أطباء الجهاز الهضمي|دكتور أسنانأخصائي الأنف والحنجرة|طبيب نساء وولادة", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bصحة\b|\bالطب|طب|دواء", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bالصحة_العقلية\b|\حمية غذائية|حمية", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bشكرًا|شكرًا|شكر|شكر|جيد|جيد|حسنًا|موافق|حسنًا", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: arabic_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bالتعيين|ميعاد\b|\bطبيب|دكتور|أطباء|أطباء القلب|أطباء الجهاز الهضمي|دكتور أسنانأخصائي الأنف والحنجرة|طبيب نساء وولادة", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bصحة\b|\bالطب|طب|دواء", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bالصحة_العقلية\b|\حمية غذائية|حمية", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bشكرًا|شكرًا|شكر|شكر|جيد|جيد|حسنًا|موافق|حسنًا", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: arabic_transform,
            TRANSITIONS: {"node2": cnd.regexp(r"\bالتعيين|ميعاد\b|\bطبيب|دكتور|أطباء|أطباء القلب|أطباء الجهاز الهضمي|دكتور أسنانأخصائي الأنف والحنجرة|طبيب نساء وولادة", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bصحة\b|\bالطب|طب|دواء", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bالصحة_العقلية\b|\حمية غذائية|حمية", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bشكرًا|شكرًا|شكر|شكر|جيد|جيد|حسنًا|موافق|حسنًا", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welcome Sir/Mam",
            TRANSITIONS: {"node2": cnd.regexp(r"\bالتعيين|ميعاد\b|\bطبيب|دكتور|أطباء|أطباء القلب|أطباء الجهاز الهضمي|دكتو أسنانأخصائي الأنف والحنجرة|طبيب نساء وولادة", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bصحة\b|\bالطب|طب|دواء", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bالصحة_العقلية\b|\حمية غذائية|حمية", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bشكرًا|شكرًا|شكر|شكر|جيد|جيد|حسنًا|موافق|حسنًا", re.IGNORECASE),
                         "node6": cnd.regexp(r"\b|لا|لا شيء|لا شيء", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "'بالتأكيد سيدي ، شكرا لزيارتك ولكن يمكنك الذهاب إلى خدمات أخرى إذا لزم الأمر يمكنك كتابة 'أهلين", 
                  TRANSITIONS: {"node1": cnd.regexp(r"\bأهلا\b|مهلا|أهلين|سلام", re.IGNORECASE)}
          },
        
  }
    
}

# An actor is an object that processes user input replicas and returns responses
# To create the actor, you need to pass the script of the dialogue `plot`
# And pass the initial node `start_label`
# and the node to which the actor will go in case of an error `fallback_label`
# If `fallback_label` is not set, then its value becomes equal to `start_label` by default
actor = Actor(plot, start_label=("greeting_flow", "start_node"), fallback_label=("greeting_flow", "fallback_node"))


# turn_handler - a function is made for the convenience of working with an actor
def turn_handler(
    in_request: str, ctx: Union[Context, str, dict], actor: Actor, true_out_response: Optional[str] = None
):
    # Context.cast - gets an object type of [Context, str, dict] returns an object type of Context
    ctx = Context.cast(ctx)
    # Add in current context a next request of user
    ctx.add_request(in_request)
    # pass the context into actor and it returns updated context with actor response
    ctx = actor(ctx)
    # get last actor response from the context
    out_response = ctx.last_response
    # the next condition branching needs for testing
    if true_out_response is not None and true_out_response != out_response:
        msg = f"in_request={in_request} -> true_out_response != out_response: {true_out_response} != {out_response}"
        raise Exception(msg)
    else:
        logging.info(f"in_request={in_request} ->\n {out_response}")
    return out_response, ctx


# testing
testing_dialog = [
    ("hallo", "Hallo!, ik ben Chatbot-assistent voor gezondheid en geneeskunde, hoe kan u helpen?"),  # start_node -> node1
    ("Ik wil een afspraak boeken?","OKE! Ik heb een afspraak gemaakt met de dokter voor gezondheidscontrole voor u, mijnheer"),
    ("ik wil een arts", "Er zijn veel artsen zoals de volgende:\n- Artsen\n- Cardiologen\n- Gastro-enterologen\n- Tandarts\n- KNO-arts\n- Gynaecoloog\nSchrijf het type arts op volgens bovenstaande manier"),    
    ("Ik wil echt een artsen", "OKE! Ik heb een afspraak gemaakt met artsen Dokter voor u meneer"),
    ("Met spoed een gastro-enterologen nodig", "OKE! Ik heb een afspraak gemaakt met gastro-enterologen Dokter voor u meneer"),
    ("Ik heb een probleem in mijn oor, kan een KNO-arts boeken", "OKE! Ik heb een afspraak gemaakt met kno-arts Dokter voor u meneer"),
    ("Ik wil een tandarts voor mijn tanden", "OKE! Ik heb een afspraak gemaakt met tandarts Dokter voor u meneer"),
    ("Ik heb een hartprobleem, boek een cardiologen", "OKE! Ik heb een afspraak gemaakt met cardiologen Dokter voor u meneer"),
    ("bedankt", "Welkom meneer/mama"),  
    ("nee", "Natuurlijk meneer, ik dank u voor uw bezoek, maar u kunt indien nodig naar andere diensten gaan, u kunt 'hallo' typen"),
    ("Hoi", "Hallo!, ik ben Chatbot-assistent voor gezondheid en geneeskunde, hoe kan u helpen?"),
    ("Wat is gezondheid?", "Gezondheid verwijst naar de mate van iemands fysieke, mentale en sociale welzijn. Deze definitie, ontleend aan de behandeling van gezondheid door de Wereldgezondheidsorganisatie, benadrukt dat gezondheid een complex concept is dat niet alleen betrekking heeft op de gezondheid van iemands lichaam, maar ook op de gemoedstoestand van een persoon en de kwaliteit van de sociale omgeving waarin hij of zij zich bevindt. leeft. De kwaliteit van de sociale omgeving kan op zijn beurt de fysieke en mentale gezondheid van een persoon beïnvloeden, wat het belang van sociale factoren voor deze tweeledige aspecten van ons algehele welzijn onderstreept."),  
    ("Definieer een geneeskunde","Geneeskunde is de sociale instelling die zowel ziekte wil voorkomen, diagnosticeren en behandelen als de gezondheid zoals zojuist gedefinieerd bevordert. De ontevredenheid over het medische establishment is toegenomen. Een deel van deze ontevredenheid komt voort uit stijgende kosten voor de gezondheidszorg en wat velen beschouwen als ongevoelige gierigheid door de zorgverzekeringsindustrie, zoals de strijd om de hervorming van de gezondheidszorg in 2009 illustreerde. Een deel van de ontevredenheid weerspiegelt ook een groeiende opvatting dat de sociale en zelfs spirituele gebieden van het menselijk bestaan een sleutelrol spelen bij gezondheid en ziekte. Deze visie heeft geleid tot een hernieuwde belangstelling voor alternatieve geneeswijzen. We komen later terug op deze vele kwesties voor de sociale instelling van de geneeskunde."),        #
    ("wat is mentale_gezondheid?", "Geestelijke gezondheid verwijst naar cognitief, gedrags- en emotioneel welzijn. Het gaat erom hoe mensen denken, voelen en zich gedragen. Mensen gebruiken de term 'geestelijke gezondheid' soms om de afwezigheid van een psychische stoornis aan te duiden. Geestelijke gezondheid kan het dagelijks leven, relaties en lichamelijke gezondheid beïnvloeden. Geestelijke gezondheid als een toestand van welzijn waarin het individu zijn of haar eigen capaciteiten realiseert, de normale stress van het leven aankan, productief en vruchtbaar kan werken en een bijdrage kan leveren aan zijn of haar gemeenschap."), 
    ("hoe kan ik mijn eetpatroon veranderen?", "Dieet is de som van voedsel dat door een persoon of een ander organisme wordt geconsumeerd. Het woord dieet impliceert vaak het gebruik van een specifieke inname van voeding om redenen van gezondheid of gewichtsbeheersing (waarbij beide vaak gerelateerd zijn). Hoewel mensen alleseters zijn, heeft elke cultuur en elke persoon bepaalde voedselvoorkeuren of voedseltaboes. Dit kan te wijten zijn aan persoonlijke smaak of ethische redenen. Individuele voedingskeuzes kunnen min of meer gezond zijn. Dieet omvat een verscheidenheid aan plantaardige en dierlijke voedingsmiddelen die het lichaam van voedingsstoffen voorzien. Dergelijke voedingsstoffen voorzien het lichaam van energie en houden het draaiende. Voedingsstoffen helpen bij het opbouwen en versterken van botten, spieren en pezen en reguleren ook lichaamsprocessen (d.w.z. bloeddruk). Water is essentieel voor groei, voortplanting en een goede gezondheid. Macronutriënten worden in relatief grote hoeveelheden geconsumeerd en omvatten eiwitten, koolhydraten en vetten en vetzuren. Micronutriënten – vitamines en mineralen – worden in relatief kleinere hoeveelheden geconsumeerd, maar zijn essentieel voor lichaamsprocessen."),
    ("ik wil een arts", "Er zijn veel artsen zoals de volgende:\n- Artsen\n- Cardiologen\n- Gastro-enterologen\n- Tandarts\n- KNO-arts\n- Gynaecoloog\nSchrijf het type arts op volgens bovenstaande manier"),    
    ("Bedankt", "Welkom meneer/mama"),
    ("Goed service", "Welkom meneer/mama"),
    ("Niets","Natuurlijk meneer, ik dank u voor uw bezoek, maar u kunt indien nodig naar andere diensten gaan, u kunt 'hallo' typen"),
]


def run_test():
    ctx = {}
    for in_request, true_out_response in testing_dialog:
        _, ctx = turn_handler(in_request, ctx, actor, true_out_response=true_out_response)


# interactive mode
def run_interactive_mode(actor):
    ctx = {}
    while True:
        in_request = input("type your answer: ")
        _, ctx = turn_handler(in_request, ctx, actor)


if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s-%(name)15s:%(lineno)3s:%(funcName)20s():%(levelname)s - %(message)s", level=logging.INFO
    )
    run_test()
    print("<-- Test Successful -->")



2022-01-27 16:04:33,211-           root:383:        turn_handler():INFO - in_request=hallo ->
 Hallo!, ik ben Chatbot-assistent voor gezondheid en geneeskunde, hoe kan u helpen?
2022-01-27 16:04:33,227-           root:383:        turn_handler():INFO - in_request=Ik wil een afspraak boeken? ->
 OKE! Ik heb een afspraak gemaakt met de dokter voor gezondheidscontrole voor u, mijnheer
2022-01-27 16:04:33,251-           root:383:        turn_handler():INFO - in_request=ik wil een arts ->
 Er zijn veel artsen zoals de volgende:
- Artsen
- Cardiologen
- Gastro-enterologen
- Tandarts
- KNO-arts
- Gynaecoloog
Schrijf het type arts op volgens bovenstaande manier
2022-01-27 16:04:33,281-           root:383:        turn_handler():INFO - in_request=Ik wil echt een artsen ->
 OKE! Ik heb een afspraak gemaakt met artsen Dokter voor u meneer
2022-01-27 16:04:33,311-           root:383:        turn_handler():INFO - in_request=Met spoed een gastro-enterologen nodig ->
 OKE! Ik heb een afspraak gemaakt m

<-- Test Successful -->


In [7]:
import re
request = "Met spoed een Gastro-enterologen nodig"
re.search(r'\bArtsen|Cardiologen|Gastro-enterologen|Tandarts|KNO-arts|Gynaecoloog', request, re.IGNORECASE)[0]

'Gastro-enterologen'